### Write all the filepaths into a single python file
---

All the paths that will be used are defined here, so that code bugs due to mismatch in paths is avoided.

In [1]:
%%writefile {'filepaths.py'}

import os

_constants_module_file = 'constants.py'
_features_module_file = 'features.py'
_transform_module_file = 'transform.py'
_trainer_module_file = 'trainer.py'
_tuner_module_file = 'tuner.py'
_pipeline_file = 'pipeline.py'
_input_base = os.path.join(*[os.pardir, 'data', 'raw'])
_segregated_base = os.path.join(*[os.pardir, 'data', 'segregated'])
_file_name = 'dataset_40_sonar.csv'
_runner_file = 'local_runner.py'
_util_file = 'util.py'
_pipeline_name = 'RP_NVIDIA_Deep_Learning_Qns1'
_pipeline_root = os.path.join('artifacts', _pipeline_name)
_metadata_path = os.path.join(*['metadata', _pipeline_name, 'metadata.db'])
_serving_model_dir = os.path.join(*[os.pardir, 'models', _pipeline_name])
_train_file = os.path.join(_segregated_base, 'train_data.csv')
_val_file = os.path.join(_segregated_base, 'val_data.csv')
_test_file = os.path.join(_segregated_base, 'test_data.csv')


Writing filepaths.py


Import the file that was just created

In [2]:
import filepaths
import numpy as np

Segregate data into train/validation/test split, and generate the `pandas-profiling` report on the train data. Tensorflow Extended does not have the functionality to do shuffled splits with the `stratify` option in sklearn `train_test_split`, hence we will use sklearn to split the data instead.

In [3]:
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
import pandas as pd
from pandas_profiling import ProfileReport
import filepaths

datapath = os.path.join(filepaths._input_base, filepaths._file_name)

train_path = os.path.join(filepaths._segregated_base, 'train_data.csv')
val_path = os.path.join(filepaths._segregated_base, 'val_data.csv')
test_path = os.path.join(filepaths._segregated_base, 'test_data.csv')

profilereport_path = 'profile_report.html'

random_state = 42
test_size = 0.2
val_size = 0.2

X_raw = pd.read_csv(datapath, sep=',')
y_raw = X_raw.pop('Class')

X_train, X_test, y_train, y_test = train_test_split(
                                        X_raw,
                                        y_raw,
                                        shuffle=True, 
                                        random_state=random_state, 
                                        test_size=test_size,
                                        stratify=y_raw)

X_train, X_val, y_train, y_val = train_test_split(
                                    X_train,
                                    y_train,
                                    shuffle=True, 
                                    random_state=random_state, 
                                    test_size=val_size/(1-test_size),
                                    stratify=y_train)

os.makedirs(filepaths._segregated_base, exist_ok=True)

X_train['Class'] = y_train
X_val['Class'] = y_val
X_test['Class'] = y_test

X_train.to_csv(filepaths._train_file, sep=',', index=False)
X_val.to_csv(filepaths._val_file, sep=',', index=False)
X_test.to_csv(filepaths._test_file, sep=',', index=False)

2022-02-24 14:18:56.622188: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-24 14:18:56.622228: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


There are a total of 60 independent variables and 1 target variable. To save time, we can use `pandas-profiling` to automatically generate a report to perform our EDA.  

The report might take some time to generate, but it is still preferable compared to the alternative of coding 60 x (boxplots, distribution graphs, etc..)  

In [ ]:
profile = ProfileReport(X_train.reset_index(drop=True),
                        title="Pandas Profiling Report",
                        explorative=True,
                        progress_bar=True)

profile.to_file(profilereport_path)

### Define the python file containing all our features and labels
---

In [5]:
%%writefile {filepaths._features_module_file}

FEATURE_KEYS = [f'attribute_{x}' for x in range(1, 61)]

LABEL_KEY = 'Class'

def transformed_name(key):

    '''
    Appends _xf to the transformed feature keys to avoid naming clash 
    between transformed and nontransformed features.
    '''
    return key + '_xf'

Writing features.py


### Define the python file containing the preprocessing code
---

In [6]:
%%writefile {filepaths._transform_module_file}

import features
import tensorflow as tf
import tensorflow_transform as tft

def _fill_in_missing(x):

    '''
    Boilerplate code to convert sparse tensors to dense tensors
    '''

    if not isinstance(x, tf.sparse.SparseTensor):
      return x

    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
        tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
            default_value),
        axis=1)

def preprocessing_fn(inputs):

    ''''
    This is the function that the tfx Transform component will run to preprocess the data
    '''

    outputs = {}

    # Standard scaling for the independet variables
    for key in features.FEATURE_KEYS:
        outputs[features.transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))

    # Label encoding for the labels
    outputs[features.transformed_name(features.LABEL_KEY)] = tf.cast(tf.equal(inputs[features.LABEL_KEY], 'Mine'), tf.int64)

    return outputs

Writing transform.py


### Define the python file containing all our hyperparameters
---

In [7]:
%%writefile {filepaths._constants_module_file}

TRAIN_HASH = 6
EVAL_HASH = 2
TEST_HASH = 2

HIDDEN_LAYER_UNITS = 8
OUTPUT_LAYER_UNITS = 2
NUM_LAYERS = 2
LEARNING_RATE = 1e-2

TRAIN_BATCH_SIZE = 20
EVAL_BATCH_SIZE = 10

TRAINING_STEPS = 1000
VALIDATION_STEPS = 100

# This is to define the lowest accuracy we can accept
LOWER_BOUND = 0.55

Writing constants.py


### Define the python file containing our training script
---

In [8]:
%%writefile {filepaths._trainer_module_file}

import os
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform as tft
from tfx import v1 as tfx
import constants
import features
from tensorflow.keras.callbacks import TensorBoard

from typing import List
from tfx_bsl.public import tfxio
from absl import logging

import keras_tuner as kt
from kerastuner.engine import base_tuner
from typing import NamedTuple, Dict, Text, Any
from tfx.components.trainer.fn_args_utils import FnArgs

TunerFnResult = NamedTuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                            ('fit_kwargs', Dict[Text, Any])])

def _get_hyperparameters() -> kt.HyperParameters:
    
    hp = kt.HyperParameters()
    hp.Int('hidden_layers', min_value=1, max_value=8, default=4)
    hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp.Int(f'units', min_value=4, max_value=16, step=2)

    return hp

def _input_fn(file_pattern: List[str], 
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int) -> tf.data.Dataset:
    
    return data_accessor.tf_dataset_factory(
                file_pattern,
                tfxio.TensorFlowDatasetOptions(
                    batch_size=batch_size,
                    label_key=features.transformed_name(features.LABEL_KEY)),
                tf_transform_output.transformed_metadata.schema).repeat()

def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        # feature_spec.pop(features.LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn

def _build_keras_model(hp: kt.HyperParameters) -> tf.keras.Model:

    # Build a simple Keras Model using Functional API

    inputs = [tf.keras.Input(shape=(1,), name=features.transformed_name(key)) 
                for key in features.FEATURE_KEYS]

    outputs = tf.keras.layers.concatenate(inputs)

    for _ in range(int(hp.get('hidden_layers'))):
        units = int(hp.get('units'))
        outputs = tf.keras.layers.Dense(units, activation='relu')(outputs)
    
    outputs = tf.keras.layers.Dense(constants.OUTPUT_LAYER_UNITS, activation='softmax')(outputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(hp.get('learning_rate')),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:

    logging.info('Running Tuner Component')

    tuner = kt.RandomSearch(
        _build_keras_model,
        objective=kt.Objective('val_sparse_categorical_accuracy', 'max'),
        max_trials=6,
        hyperparameters=_get_hyperparameters(),
        allow_new_entries=False,
        directory=fn_args.working_dir,
        project_name='kt_hyperband'
    )

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_dataset = _input_fn(
                    file_pattern=fn_args.train_files,
                    data_accessor=fn_args.data_accessor,
                    tf_transform_output=tf_transform_output,
                    batch_size=constants.TRAIN_BATCH_SIZE,
                )

    eval_dataset = _input_fn(
                        file_pattern=fn_args.train_files,
                        data_accessor=fn_args.data_accessor,
                        tf_transform_output=tf_transform_output,
                        batch_size=constants.EVAL_BATCH_SIZE,
                    )


    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            'x': train_dataset,
            'validation_data': eval_dataset,
            'steps_per_epoch': fn_args.train_steps,
            'validation_steps': fn_args.eval_steps,
        }
    )

def run_fn(fn_args: tfx.components.FnArgs):

    logging.info('Running Transform Component')

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(
                        file_pattern=fn_args.train_files,
                        data_accessor=fn_args.data_accessor,
                        tf_transform_output=tf_transform_output,
                        batch_size=constants.TRAIN_BATCH_SIZE,
                    )

    eval_dataset = _input_fn(
                        file_pattern=fn_args.train_files,
                        data_accessor=fn_args.data_accessor,
                        tf_transform_output=tf_transform_output,
                        batch_size=constants.EVAL_BATCH_SIZE,
                    )
    
    if fn_args.hyperparameters:
        hp = kt.HyperParameters.from_config(fn_args.hyperparameters)
    else:
        hp = _get_hyperparameters()

    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = _build_keras_model(hp)

    model.summary()

    tensorboard_callback = TensorBoard(
                                log_dir=fn_args.model_run_dir,
                                update_freq='epoch'
                            )

    # TODO: Create a custom callback to generate confusion matrix
    # Add callback to save checkpoints regularly

    callbacks = [tensorboard_callback,]

    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps,
        callbacks=callbacks
    )

    signatures = {'serving_default':
                        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                            tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
                        ),
                }

    # Save model in Tensorflow SavedModel format
    model.save(
        fn_args.serving_model_dir, 
        save_format='tf',
        signatures=signatures)

Writing trainer.py


### Run the pipeline interactively
---

Imports

In [9]:
import features
import constants
import filepaths

import os
import tensorflow as tf

import pandas as pd

from tfx import v1 as tfx
from tfx.v1.components import CsvExampleGen 
from tfx.v1.components import StatisticsGen 
from tfx.v1.components import SchemaGen
from tfx.v1.components import ExampleValidator
from tfx.v1.components import Transform
from tfx.v1.components import Trainer
from tfx.v1.components import Evaluator
from tfx.v1.components import Pusher
from tfx.v1.components import Tuner

from tfx.v1 import proto
from tfx.proto import trainer_pb2

import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

Create an interactive context to run the pipline in a notebook

In [10]:
context = InteractiveContext()

Load the train and validation data using TFX CsvExampleGen component

In [11]:
# Component to load the data
input = proto.Input(splits=[
    proto.Input.Split(name='train', pattern='train_data.csv'),
    proto.Input.Split(name='eval', pattern='val_data.csv'),
])   

example_gen = CsvExampleGen(input_base=filepaths._segregated_base, input_config=input)
context.run(example_gen, enable_cache=False)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:train,num_files:1,total_bytes:52478,xor_checksum:1645683538,sum_checksum:1645683538\nsplit:eval,num_files:1,total_bytes:18305,xor_checksum:1645683538,sum_checksum:1645683538"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Generate the statistics using TFX StatisticsGen component

In [12]:
# Component to generate the statistics
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=False)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 16
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        base_type: STATISTICS
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

It is possible to perform EDA by using the statistics generated by TFX StatisticsGen.  
This is much faster than using `pandas-profiling`, but has fewer features as well.

In [13]:
# Run cell to view the statistics for the train and validation data
context.show(statistics_gen.outputs['statistics'])

The data here is so clean there really isn't much to do except to use standardization on the independent variables. More information on what the 60 independent variables represents are needed to understand if we need to perform feature engineering. For now we will leave it as is and just standardise the 60 independent variables.

Generate the data schema using TFX SchemaGen component.  

In [14]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True
)
context.run(schema_gen, enable_cache=False)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

It is possible to use user defined schema as well, but the data is so clean and well formatted  
that we can let TFX infer the schema.  

Just to be sure, we can view the schema generated as well.

In [15]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Class',STRING,required,,'Class'
'attribute_1',FLOAT,required,,-
'attribute_10',FLOAT,required,,-
'attribute_11',FLOAT,required,,-
'attribute_12',FLOAT,required,,-
...,...,...,...,...
'attribute_6',FLOAT,required,,-
'attribute_60',FLOAT,required,,-
'attribute_7',FLOAT,required,,-


,Values
Domain,
'Class',"'Mine', 'Rock'"


ExampleValidator uses the statistics generated to check against the schema for anomalies in the data. 

In [16]:
# Component to validate the data
example_validator = ExampleValidator(
            statistics=statistics_gen.outputs['statistics'],
            schema=schema_gen.outputs['schema']
        )
context.run(example_validator, enable_cache=False)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 20
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

This is so much faster than handcoding data validation using tools like `pytest`

In [17]:
context.show(example_validator.outputs['anomalies'])

Preprocessing of data is handled by TFX Transform component.  
It uses the transform module file we defined earlier, and calls the `prepreocessing_fn` function defined in the module file against the data.

In [18]:
transform = Transform(
                examples=example_gen.outputs['examples'],
                schema=schema_gen.outputs['schema'],
                module_file=filepaths._transform_module_file
            )
            
context.run(transform, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying pipeline.py -> build/lib
copying trainer.py -> build/lib
copying features.py -> build/lib
copying constants.py -> build/lib
copying local_runner.py -> build/lib
copying filepaths.py -> build/lib
installing to /tmp/tmp6qrq5q2j
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmp6qrq5q2j
copying build/lib/pipeline.py -> /tmp/tmp6qrq5q2j
copying build/lib/trainer.py -> /tmp/tmp6qrq5q2j
copying build/lib/features.py -> /tmp/tmp6qrq5q2j
copying build/lib/constants.py -> /tmp/tmp6qrq5q2j
copying build/lib/local_runner.py -> /tmp/tmp6qrq5q2j
copying build/lib/filepaths.py -> /tmp/tmp6qrq5q2j
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_cod

2022-02-24 14:27:43.401072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-24 14:27:43.401167: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-24 14:27:43.401214: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-DEIQ1F4): /proc/driver/nvidia/version does not exist
2022-02-24 14:27:43.401534: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Use ref() instead.
2022-02-24 14:28:11.810965: W tensorflow/python/util/

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Transform/transform_graph/5/.temp_path/tftransform_tmp/7a872cb458684d089b474de1833693a0/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Transform/transform_graph/5/.temp_path/tftransform_tmp/7a872cb458684d089b474de1833693a0/assets


INFO:tensorflow:tensorflow_text is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 22
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 22
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"eval\", \"train\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        base_type: DATASET
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 23
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 18
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
           

To perform hyperparameter tuning, we use TFX Tuner component to search for the best 
1. number of hidden layers, 
2. units in each hidden layer, and
3. learning rate  

The component calls the `tuner_fn` function in the module file given.  

In [19]:
tuner = Tuner(
    module_file=filepaths._trainer_module_file,  
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=20),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=5))
    
context.run(tuner, enable_cache=False)

Trial 6 Complete [00h 00m 01s]
val_sparse_categorical_accuracy: 0.7400000095367432

Best val_sparse_categorical_accuracy So Far: 0.9399999976158142
Total elapsed time: 00h 00m 09s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in /tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/.temp/6/kt_hyperband
Showing 10 best trials
Objective(name='val_sparse_categorical_accuracy', direction='max')
Trial summary
Hyperparameters:
hidden_layers: 3
learning_rate: 0.01
units: 12
Score: 0.9399999976158142
Trial summary
Hyperparameters:
hidden_layers: 3
learning_rate: 0.01
units: 14
Score: 0.8600000143051147
Trial summary
Hyperparameters:
hidden_layers: 7
learning_rate: 0.01
units: 4
Score: 0.8600000143051147
Trial summary
Hyperparameters:
hidden_layers: 3
learning_rate: 0.001
units: 14
Score: 0.7400000095367432
Trial summary
Hyperparameters:
hidden_layers: 3
learning_rate: 0.001
units: 8
Score: 0.699999988079071
Trial summary
Hyperparameters:
hidden_layers: 2
learning_rate: 0.0001
units: 6
Score: 0.4399999976158142


ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 13
        type_id: 25
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Tuner/best_hyperparameters/6"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 25
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

TFX Trainer component takes in the transformed data, the best hyperparameters and the training module file to perform model training.  

The component calls the `run_fn` function in the module file given.  

In [20]:
trainer = Trainer(
            module_file=filepaths._trainer_module_file,
            examples=transform.outputs['transformed_examples'],
            transform_graph=transform.outputs['transform_graph'],
            schema=schema_gen.outputs['schema'],
            hyperparameters = tuner.outputs['best_hyperparameters'],
            train_args=tfx.proto.TrainArgs(num_steps=constants.TRAINING_STEPS),
            eval_args=tfx.proto.EvalArgs(num_steps=constants.VALIDATION_STEPS),
        )
        
context.run(trainer, enable_cache=False)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform.py -> build/lib
copying pipeline.py -> build/lib
copying trainer.py -> build/lib
copying features.py -> build/lib
copying constants.py -> build/lib
copying local_runner.py -> build/lib
copying filepaths.py -> build/lib
installing to /tmp/tmp8lh80jlb
running install
running install_lib
copying build/lib/transform.py -> /tmp/tmp8lh80jlb
copying build/lib/pipeline.py -> /tmp/tmp8lh80jlb
copying build/lib/trainer.py -> /tmp/tmp8lh80jlb
copying build/lib/features.py -> /tmp/tmp8lh80jlb
copying build/lib/constants.py -> /tmp/tmp8lh80jlb
copying build/lib/local_runner.py -> /tmp/tmp8lh80jlb
copying build/lib/filepaths.py -> /tmp/tmp8lh80jlb
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trai

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attribute_1_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 attribute_2_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 attribute_3_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 attribute_4_xf (InputLayer)    [(None, 1)]          0           []                               
                                                                                            

2022-02-24 14:29:20.521777: W tensorflow/core/framework/dataset.cc:744] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


...

INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.
/home/weishengtoh/miniconda3/envs/dl_assignment/lib/python3.8/site-packages/keras/engine/functional.py:559: UserWarning: Input dict contained keys ['Class_xf'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Trainer/model/7/Format-Serving/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Trainer/model/7/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 14
        type_id: 27
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 27
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 15
        type_id: 28
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Model evaluation is performed using TFX Evaluator component.  

The component takes in: 
1. `tfma.ModelSpec` which contains the location of the model and signature used for predictions 
2. `tfma.MetricsSpec` which is used to cinfigure the metrics that will be calculated
3. `tfma.SlicingSpec` which is used to define the slices - this is perhaps more useful for more complicated projects where we need to assess how well the model is doing for different groups of people etc. Empty slices simply uses the entire model.

Define the `tfma.ModelSpec`

In [21]:
model_specs = [tfma.ModelSpec(signature_name='serving_default', 
                            preprocessing_function_names=['tft_layer'],
                            label_key=features.transformed_name(features.LABEL_KEY))]

Define the `tfma.MetricsSpec`

In [22]:
metrics_specs = [tfma.MetricsSpec(metrics=[
                                    tfma.MetricConfig(class_name='AUC'),
                                    tfma.MetricConfig(class_name='ConfusionMatrixPlot'),
                                    tfma.MetricConfig(class_name='SparseCategoricalCrossentropy'),
                                    tfma.MetricConfig(
                                        class_name='SparseCategoricalAccuracy',
                                        threshold=tfma.MetricThreshold(
                                            value_threshold=tfma.GenericValueThreshold(lower_bound={'value': constants.LOWER_BOUND}),
                                            change_threshold=tfma.GenericChangeThreshold(
                                                direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                                absolute={'value': -1e-10})))
                                    ]      
                                ),
                ]

Define the `tfma.SlicingSpec`

In [23]:
slicing_specs = [tfma.SlicingSpec()]

To compare against a baseline model, we need to feed the Resolver component into TFX Evaluator component as well.  
The Resolver component search for the latest 'blessed' model in TFX ML Metadata and uses it as the baseline.  
'Blessed' is the term used to defined a model that the TFX Evaluator has passed.

In [24]:
model_resolver = tfx.dsl.Resolver(
                    strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
                    model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
                    model_blessing=tfx.dsl.Channel(
                        type=tfx.types.standard_artifacts.ModelBlessing
                    )
                ).with_id('latest_blessed_model_resolver')
context.run(model_resolver, enable_cache=False)

ExecutionResult(
    component_id: latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

Here the `tfma.ModelSpec`, `tfma.MetricsSpec`, `tfma.SlicingSpec` and `tfx.dsl.Resolver` are fed into the TFX Evaluator component. These are used to evaluate against the trained model.  

In [25]:
eval_config = tfma.EvalConfig(
    model_specs=model_specs,
    slicing_specs=slicing_specs,
    metrics_specs=metrics_specs,
)

evaluator = Evaluator(
                    examples=example_gen.outputs['examples'],
                    model=trainer.outputs['model'],
                    baseline_model=model_resolver.outputs['model'],
                    eval_config=eval_config)
                    
context.run(evaluator, enable_cache=False)


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f364cf946a0> and <keras.engine.input_layer.InputLayer object at 0x7f369687b820>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f364cf946a0> and <keras.engine.input_layer.InputLayer object at 0x7f369687b820>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f369521b160> and <keras.engine.input_layer.InputLayer object at 0x7f3695345cd0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f369521b160> and <keras.engine.input_layer.InputLayer object at 0x7f3695345cd0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f3695248850> and <keras.engine.input_layer.InputLayer object at 0x7f364fb67c70>).


...


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f378723ebe0> and <keras.engine.input_layer.InputLayer object at 0x7f364e6120d0>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f378723ebe0> and <keras.engine.input_layer.InputLayer object at 0x7f364e6120d0>).


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 16
        type_id: 31
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Evaluator/evaluation/9"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 31
        name: "ModelEvaluation"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 17
        type_id: 32
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Evaluator/blessing/9"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Trainer/model/7"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 14
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 32
        name: "ModelBlessing"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

The final component in TFX pipeline is the Pusher component.  
The component checks and exports the model to a SavedModel format if the model has passed the Evaluator component.

In [26]:
pusher = Pusher(
            model=trainer.outputs['model'],
            model_blessing=evaluator.outputs['blessing'],
            push_destination=tfx.proto.PushDestination(
                filesystem=tfx.proto.PushDestination.Filesystem(
                    base_directory=filepaths._serving_model_dir
                )
            )
        )
        
context.run(pusher, enable_cache=False)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 18
        type_id: 34
        uri: "/tmp/tfx-interactive-2022-02-24T14_27_25.792560-h7mnqdg_/Pusher/pushed_model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "../models/RP_NVIDIA_Deep_Learning_Qns1/1645684191"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1645684191"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.5.0"
          }
        }
        state: LIVE
        , artifact_type: id: 34
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Inferencing
---

CsvExampleGen was used to automatically convert our train and validation dataset to a suitable format for TFX.  

Here we need to manually convert our test data to be used by the model.  

In [27]:
FEATURE_KEYS = [f'attribute_{x}' for x in range(1, 61)]
LABEL_KEY = 'Class'

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(str(value), encoding="raw_unicode_escape")]))

def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _example_generator(df):
    examples = []
    for index, row in df.iterrows():

        features = {}
        for key in FEATURE_KEYS:
            features[key] = _float_feature(row[key])
        features[LABEL_KEY] = _bytes_feature(row[LABEL_KEY])

        example_proto = tf.train.Example(features=tf.train.Features(feature=features))
        examples.append(example_proto.SerializeToString())

    return examples

Here we retrieve the latest model for inferencing

In [28]:
def get_inference_fn(model_directory):
    model_directories = (d for d in os.scandir(model_directory) if d.is_dir())
    model_path = max(model_directories, key=lambda i: int(i.name)).path

    loaded_model = tf.keras.models.load_model(model_path)

    return loaded_model.signatures["serving_default"]

In [29]:
df = pd.read_csv(filepaths._test_file, sep=',')

test_labels = df[['Class']]
df_results = test_labels.rename(columns={'Class': 'actual_label'})

inference_fn = get_inference_fn('../models/RP_NVIDIA_Deep_Learning_Qns1')
results = inference_fn(examples=tf.constant(_example_generator(df)))

activated_logits = results['output_0'].numpy()
pred = np.argmax(activated_logits, axis=1)


Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f364c684490> and <keras.engine.input_layer.InputLayer object at 0x7f3630345910>).



Two checkpoint references resolved to different objects (<keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7f364c684490> and <keras.engine.input_layer.InputLayer object at 0x7f3630345910>).


We encoded 'Mine' as 1 and 'Rock' as 0 previously, and this information is required to decode the predictions for the test dataset

In [30]:
labels_map = {1: 'Mine', 0: 'Rock'}

df_results['pred'] = pred
df_results['pred_label'] = df_results['pred'].map(labels_map)
df_results['correct'] = df_results['pred_label'] == df_results['actual_label'] 
df_results.head()


,actual_label,pred,pred_label,correct
0,Rock,0,Rock,True
1,Rock,1,Mine,False
2,Rock,0,Rock,True
3,Mine,1,Mine,True
4,Mine,1,Mine,True


In [31]:
df_results['correct'].value_counts()

True     37
False     5
Name: correct, dtype: int64

Obtain the test metrics

In [32]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

accuracy = accuracy_score(df_results['actual_label'], df_results['pred_label'])
f1 = f1_score(df_results['actual_label'], df_results['pred_label'], pos_label='Mine')
precision = precision_score(df_results['actual_label'], df_results['pred_label'], pos_label='Mine')
recall = recall_score(df_results['actual_label'], df_results['pred_label'], pos_label='Mine')

print(f'Test Accuracy: {accuracy:0.2f}')
print(f'Test f1_score: {f1:0.2f}')
print(f'Test precision: {precision:0.2f}')
print(f'Test recall: {recall:0.2f}')

Test Accuracy: 0.88
Test f1_score: 0.89
Test precision: 0.84
Test recall: 0.95


Plot and save the confusion matrix

In [33]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm_display = ConfusionMatrixDisplay.from_predictions(df_results['actual_label'], df_results['pred_label'])
cm_display.figure_.savefig("confusionmatrix.png")

Plot and save the ROC Curve

In [34]:
from sklearn.metrics import RocCurveDisplay

roc_display = RocCurveDisplay.from_predictions(
                                    df_results['actual_label'], 
                                    df_results['pred'], 
                                    pos_label='Mine')
roc_display.figure_.savefig("roc_curve.png")